<a href="https://colab.research.google.com/github/Nuha4/Algorithms/blob/master/metrocard_cpi_fuel_correlation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr, spearmanr
import numpy as np

In [ ]:
# ---------- Step 2: Load and Clean CPI Data ----------
cpi_preview = pd.read_excel("CPI_640101.xlsx", sheet_name='Data1', header=None)

# Extract relevant columns: Date, Index, YoY %, QoQ %
# cpi_data = cpi_preview.iloc[10:, [0, 4, 13, 22]].copy()
# cpi_data.columns = ['Date', 'CPI_Adelaide', 'YoY_Change_Adelaide', 'QoQ_Change_Adelaide']

cpi_data = cpi_preview.iloc[10:, [0, 4]].copy()
cpi_data.columns = ['Date', 'CPI_Adelaide']

# Convert to datetime and numeric
cpi_data['Date'] = pd.to_datetime(cpi_data['Date'], errors='coerce')
cpi_data['CPI_Adelaide'] = pd.to_numeric(cpi_data['CPI_Adelaide'], errors='coerce')
# cpi_data['YoY_Change_Adelaide'] = pd.to_numeric(cpi_data['YoY_Change_Adelaide'], errors='coerce')
# cpi_data['QoQ_Change_Adelaide'] = pd.to_numeric(cpi_data['QoQ_Change_Adelaide'], errors='coerce')

# Drop rows with missing dates
cpi_data = cpi_data[cpi_data['Date'].notna()]

# Filter for 2022–2024
cpi_data = cpi_data[(cpi_data['Date'] >= '2022-01-01') & (cpi_data['Date'] <= '2024-12-31')]

print(cpi_data.head())

          Date  CPI_Adelaide
304 2022-03-01         122.7


In [28]:

# Interpolate quarterly data to monthly
cpi_monthly = cpi_data.set_index('Date').resample('MS').interpolate(method='linear').reset_index()
cpi_monthly['Month'] = cpi_monthly['Date'].dt.to_period('M').dt.to_timestamp()

print(cpi_monthly)

        Date  CPI_Adelaide      Month
0 2022-03-01         122.7 2022-03-01


In [30]:

# ---------- Step 2: Load and Clean Fuel Data ----------
fuel_df = pd.read_excel("australian-petroleum-statistics-data-extract-december-2024.xlsx",
                        sheet_name='Sales by state and territory')

fuel_df = fuel_df[(fuel_df['Month'] >= '2022-01-01') & (fuel_df['Month'] <= '2024-12-31')]

# print(fuel_df.head())

fuel_df = fuel_df[fuel_df['State'] == 'SA'].copy()
fuel_df['Month'] = pd.to_datetime(fuel_df['Month'], errors='coerce')
# print(fuel_df.head())

fuel_df.replace('n.p.', np.nan, inplace=True)
fuel_df['Automotive gasoline: total (ML)'] = pd.to_numeric(fuel_df['Automotive gasoline: total (ML)'], errors='coerce')
fuel_df = fuel_df[['Month', 'Automotive gasoline: total (ML)']]
fuel_df = fuel_df.rename(columns={'Automotive gasoline: total (ML)': 'Fuel_Consumption_ML'})


print(fuel_df.head())

         Month  Fuel_Consumption_ML
660 2022-01-01                 82.0
661 2022-02-01                 82.3
662 2022-03-01                 89.8
663 2022-04-01                 84.0
664 2022-05-01                 87.2


/tmp/ipython-input-30-186062716.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fuel_df.replace('n.p.', np.nan, inplace=True)


In [35]:
# ---------- Step 3: Load and Aggregate MetroCard Data ----------
metro_df = pd.read_csv("banded-adelaide-metrocard-validations-2024-q4.csv")

metro_df['VALIDATION_DATE'] = pd.to_datetime(metro_df['VALIDATION_DATE'], dayfirst=True)

metro_df['BAND_BOARDINGS_FLOOR'] = pd.to_numeric(metro_df['BAND_BOARDINGS_FLOOR'], errors='coerce')
metro_df['Month'] = metro_df['VALIDATION_DATE'].dt.to_period('M').dt.to_timestamp()

# print(metro_df)

monthly_metro = metro_df.groupby('Month')['BAND_BOARDINGS_FLOOR'].sum().reset_index()
monthly_metro = monthly_metro.rename(columns={'BAND_BOARDINGS_FLOOR': 'MetroCard_Usage'})

monthly_metro = monthly_metro[
    (monthly_metro['Month'] >= '2022-01-01') &
    (monthly_metro['Month'] <= '2024-12-31')
]

print(monthly_metro)

       Month  MetroCard_Usage
0 2024-10-01          3003736
1 2024-11-01          2557831
2 2024-12-01          2193206


In [ ]:

# ---------- Step 4: Merge All Datasets ----------
combined_df = pd.merge(cpi_monthly[['Month', 'CPI_Adelaide']], fuel_df, on='Month', how='outer')
combined_df = pd.merge(combined_df, monthly_metro, on='Month', how='outer')
combined_df = combined_df.sort_values('Month').reset_index(drop=True)

# ---------- Final Output ----------
print(combined_df.head())  # Preview